In [1]:
!pip install gdown
!pip install pandarallel

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader 
from datasets import load_dataset, Dataset
from tqdm import tqdm
import datasets

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    TrainingArguments, Trainer, DataCollatorForSeq2Seq,AutoTokenizer,FlaxT5ForConditionalGeneration,AutoModelForSeq2SeqLM
  )


In [4]:
url='https://drive.google.com/drive/folders/1QvhirnOFoLqPZYeBz8M9oMZh-hxnOCnt?usp=sharing'

In [5]:
import gdown
gdown.download_folder(url, quiet=True)

['/kaggle/working/data_wav2vec2/benchmark.parquet',
 '/kaggle/working/data_wav2vec2/custom_data.parquet',
 '/kaggle/working/data_wav2vec2/data_sharded/test/cache-43f0f09683d0ab82.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/test/cache-a99e762e72e1079e.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/test/data-00000-of-00001.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/test/dataset_info.json',
 '/kaggle/working/data_wav2vec2/data_sharded/test/state.json',
 '/kaggle/working/data_wav2vec2/data_sharded/train/cache-bf7ee38c5d6c7cce.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/cache-c5e24c7182b4fbb4.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00000-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00001-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00002-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00003-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_shar

In [6]:
df_custom = pd.read_parquet('/kaggle/working/data_wav2vec2/custom_data.parquet')
df_wav2vec2_all = pd.read_parquet('/kaggle/working/data_wav2vec2/wav2vec2_all.parquet')
df_wav2vec2_masked = pd.read_parquet('/kaggle/working/data_wav2vec2/wav2vec2_masked.parquet')

In [7]:
df_custom

,sentence,masked_sen,error_word,correct_word,lavenshtein_distance
0,আর জামিনের আবেদন করলেন না অর্পিতা,আর জামিনের [MASK] করলেন না অর্পিতা,সবেদন,আবেদন,20
1,অর্পিতা মুখার্জি কারাগারেই,অর্পিতা মুখার্জি [MASK],কারাগ ক্ষ্ম রএই,কারাগারেই,53
2,তাঁর জীবন সংশয়ের আশঙ্কা রয়েছে,তাঁর [MASK] সংশয়ের আশঙ্কা রয়েছে,জী ন্ধ ন,জীবন,62
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন খারিজ কর...,আদালত শুনানি শেষে পার্থের জামিন আবেদন [MASK] ক...,খা র্ধ িজ,খারিজ,44
4,অর্পিতাকেও দিনের জেল হেফাজতে রাখার আদেশ দেন,অর্পিতাকেও দিনের জেল [MASK] রাখার আদেশ দেন,হে গ্ম াজতে,হেফাজতে,45
...,...,...,...,...,...
2992534,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম আলোকে...,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম [MASK...,আলোযে,আলোকে,20
2992535,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে আটক,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে [MASK],আলোযে,আটক,80
2992536,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,আলোযে,আনা,80
2992537,পাঁচটি অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,[MASK] অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,পঁচটি,পাঁচটি,16


In [8]:
df_wav2vec2_all

,path,wav2vec2,arpa_4gram,sentence,wav2vec2_label,arpa_label,wer_wav2vec2,wer_arpa
0,slr_000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশে দায়িত্ব নেবে,"['S', 'C', 'C']","['S', 'C', 'C']",33,33
1,slr_000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে,এ ধরণের কার্ড নিয়ে,"['S', 'S', 'S', 'C']","['S', 'S', 'S', 'C']",75,75
2,slr_00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে,হাসির বিষয় হয়েই আছে,"['S', 'S', 'C', 'C']","['C', 'C', 'C', 'C']",50,0
3,slr_00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে,সার্ক দেশগুলোতে,"['I', 'S', 'S']","['I', 'S', 'S']",100,100
4,slr_00012880b1.flac,শহজি জাদে,শহরে যারা,সহজেই যাওয়া যায়,"['D', 'S', 'S']","['D', 'S', 'S']",100,100
...,...,...,...,...,...,...,...,...
172779,common_voice_bn_31562650.mp3,তারপর মুহাম্মদ আহমেদের ছেলে মুহাম্মদ তৈয়ব কাসে...,তারপর মোহাম্মদ আহমেদের ছেলে মোহাম্মদ তৈয়ব কাসে...,তার পর মুহাম্মদ আহমদের ছেলে মুহাম্মদ তৈয়ব কাসে...,"['D', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'C', ...","['D', 'S', 'S', 'S', 'C', 'S', 'C', 'D', 'S', ...",46,60
172780,common_voice_bn_31685658.mp3,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসে ...,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', ...",35,35
172781,common_voice_bn_31685660.mp3,আব্দুল মালেক ময়মনসিঙহ জেলার মুক্তিযুদ্ধের সংগঠ...,আবদুল মালেক ময়মনসিংহ জেলার মুক্তিযুদ্ধের সংগঠক...,আদ্বুল মালেক ময়মনসিংহ জেলার মুক্তিযুদ্ধের সংগঠ...,"['S', 'C', 'S', 'C', 'C', 'C', 'C']","['S', 'C', 'C', 'C', 'C', 'C', 'C']",28,14
172782,common_voice_bn_31677232.mp3,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,তার জয়ের মাধ্যমে প্রথমবারের মতো কোনো তৃণমূল কং...,তার জয়ের মাধ্যমে প্রথমবারের মত কোনো তৃণমূল কংগ...,"['C', 'C', 'C', 'C', 'S', 'S', 'S', 'C', 'S', ...","['C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'S', ...",29,17


In [9]:
df_wav2vec2_masked

,wav2vec2,masked_wav2vec2,error_word,correct_word,sentence
0,হাসি বিষল হয়েই আছে,হাসি [MASK] হয়েই আছে,বিষল,বিষয়,হাসির বিষয় হয়েই আছে
1,শাক দেশ গুরুতে,শাক দেশ [MASK],গুরুতে,,সার্ক দেশগুলোতে
2,শহজি জাদে,[MASK] জাদে,শহজি,সহজেই,সহজেই যাওয়া যায়
3,শহজি জাদে,শহজি [MASK],জাদে,যাওয়া,সহজেই যাওয়া যায়
4,অন্তর্ভুক্ত করেছে রাশিয়াম,অন্তর্ভুক্ত করেছে [MASK],রাশিয়াম,রাশিয়া,অন্তর্ভুক্তও করেছে রাশিয়া
...,...,...,...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,পদর্শন,পশ্রন,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসে ...
248770,আব্দুল মালেক ময়মনসিঙহ জেলার মুক্তিযুদ্ধের সংগঠ...,আব্দুল মালেক [MASK] জেলার মুক্তিযুদ্ধের সংগঠক ...,ময়মনসিঙহ,ময়মনসিংহ,আদ্বুল মালেক ময়মনসিংহ জেলার মুক্তিযুদ্ধের সংগঠ...
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,পার্থি,পথ্রী,তার জয়ের মাধ্যমে প্রথমবারের মত কোনো তৃণমূল কংগ...
248772,এখানে ল্বযাট ব্যাং্ক পরিষেবা বর্তমান,এখানে [MASK] ব্যাং্ক পরিষেবা বর্তমান,ল্বযাট,ল্বাড,এখানে ল্বাড ব্যাঙ্ক পরিষেবা বর্তমান


In [10]:
df_custom = df_custom[['masked_sen','error_word','correct_word']]
df_custom

,masked_sen,error_word,correct_word
0,আর জামিনের [MASK] করলেন না অর্পিতা,সবেদন,আবেদন
1,অর্পিতা মুখার্জি [MASK],কারাগ ক্ষ্ম রএই,কারাগারেই
2,তাঁর [MASK] সংশয়ের আশঙ্কা রয়েছে,জী ন্ধ ন,জীবন
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন [MASK] ক...,খা র্ধ িজ,খারিজ
4,অর্পিতাকেও দিনের জেল [MASK] রাখার আদেশ দেন,হে গ্ম াজতে,হেফাজতে
...,...,...,...
2992534,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম [MASK...,আলোযে,আলোকে
2992535,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে [MASK],আলোযে,আটক
2992536,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,আলোযে,আনা
2992537,[MASK] অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,পঁচটি,পাঁচটি


In [11]:
df_wav2vec2_masked = df_wav2vec2_masked[['masked_wav2vec2','error_word','correct_word']]
df_wav2vec2_masked

,masked_wav2vec2,error_word,correct_word
0,হাসি [MASK] হয়েই আছে,বিষল,বিষয়
1,শাক দেশ [MASK],গুরুতে,
2,[MASK] জাদে,শহজি,সহজেই
3,শহজি [MASK],জাদে,যাওয়া
4,অন্তর্ভুক্ত করেছে [MASK],রাশিয়াম,রাশিয়া
...,...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,পদর্শন,পশ্রন
248770,আব্দুল মালেক [MASK] জেলার মুক্তিযুদ্ধের সংগঠক ...,ময়মনসিঙহ,ময়মনসিংহ
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,পার্থি,পথ্রী
248772,এখানে [MASK] ব্যাং্ক পরিষেবা বর্তমান,ল্বযাট,ল্বাড


In [12]:
df_custom.rename(columns = {'masked_sen':'input','correct_word':'output'},inplace=True)
df_custom

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,input,error_word,output
0,আর জামিনের [MASK] করলেন না অর্পিতা,সবেদন,আবেদন
1,অর্পিতা মুখার্জি [MASK],কারাগ ক্ষ্ম রএই,কারাগারেই
2,তাঁর [MASK] সংশয়ের আশঙ্কা রয়েছে,জী ন্ধ ন,জীবন
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন [MASK] ক...,খা র্ধ িজ,খারিজ
4,অর্পিতাকেও দিনের জেল [MASK] রাখার আদেশ দেন,হে গ্ম াজতে,হেফাজতে
...,...,...,...
2992534,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম [MASK...,আলোযে,আলোকে
2992535,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে [MASK],আলোযে,আটক
2992536,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,আলোযে,আনা
2992537,[MASK] অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,পঁচটি,পাঁচটি


In [13]:
df_wav2vec2_masked.rename(columns={'masked_wav2vec2':'input','correct_word':'output'},inplace=True)
df_wav2vec2_masked

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,input,error_word,output
0,হাসি [MASK] হয়েই আছে,বিষল,বিষয়
1,শাক দেশ [MASK],গুরুতে,
2,[MASK] জাদে,শহজি,সহজেই
3,শহজি [MASK],জাদে,যাওয়া
4,অন্তর্ভুক্ত করেছে [MASK],রাশিয়াম,রাশিয়া
...,...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,পদর্শন,পশ্রন
248770,আব্দুল মালেক [MASK] জেলার মুক্তিযুদ্ধের সংগঠক ...,ময়মনসিঙহ,ময়মনসিংহ
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,পার্থি,পথ্রী
248772,এখানে [MASK] ব্যাং্ক পরিষেবা বর্তমান,ল্বযাট,ল্বাড


In [14]:
df = df_custom.append(df_wav2vec2_masked)
df = df.dropna(how='any',axis=0) 
df

,input,error_word,output
0,আর জামিনের [MASK] করলেন না অর্পিতা,সবেদন,আবেদন
1,অর্পিতা মুখার্জি [MASK],কারাগ ক্ষ্ম রএই,কারাগারেই
2,তাঁর [MASK] সংশয়ের আশঙ্কা রয়েছে,জী ন্ধ ন,জীবন
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন [MASK] ক...,খা র্ধ িজ,খারিজ
4,অর্পিতাকেও দিনের জেল [MASK] রাখার আদেশ দেন,হে গ্ম াজতে,হেফাজতে
...,...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,পদর্শন,পশ্রন
248770,আব্দুল মালেক [MASK] জেলার মুক্তিযুদ্ধের সংগঠক ...,ময়মনসিঙহ,ময়মনসিংহ
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,পার্থি,পথ্রী
248772,এখানে [MASK] ব্যাং্ক পরিষেবা বর্তমান,ল্বযাট,ল্বাড


In [15]:
df = df.sample(2041313)
df = df.reset_index(drop=True)
df

,input,error_word,output
0,যদি এই সিদ্ধান্ত [MASK] হয়,রার্হকর,কার্যকর
1,এই প্রশ্নের কোনো জবাব [MASK] হাঙ্গেরি সরকারের ...,দেয়নউ,দেয়নি
2,তারপর গতকাল সরিয়ে [MASK] হয় কোচ সিলভারউডকেও,দ দ্ধ ওয়া,দেওয়া
3,পরিস্থিতি একটু স্বাভাবিক হতেই একের পর এক বিয়ে ...,চ্ছো,হচ্ছে
4,একটি উপসম্পাদকীয় [MASK],নিবন্ঙে,নিবন্ধে
...,...,...,...
2041308,কয়েকটি [MASK] খোলা পাম তেল বিক্রি হচ্ছে টাকা ল...,দোকড়নে,দোকানে
2041309,নিহত নারী স্বজন আমির [MASK] জানান,স্ত্রী,আলী
2041310,ফলে ভিন্ন ভাষাভাষীদের সঙ্গে [MASK] মিটিং করা যায়,স্বও্ছউ্দো,স্বচ্ছন্দে
2041311,[MASK] প্রায় ট্রাক ময়লাআবর্জনা অপসারণ করেছি,আগের,আমি


In [16]:
def input_merge(input_sen,error_word):
    return input_sen + ' , ' + error_word

In [17]:
df['input'] = df.parallel_apply(lambda x: input_merge(x.input,x.error_word) ,axis=1)

In [18]:
df.sample(5)

,input,error_word,output
904885,"[MASK] এস এম শফিউদ্দিন আহমেদ বলেন , অ দ্ম ু্ঠানে",অ দ্ম ু্ঠানে,অনুষ্ঠানে
335501,"এরপরও যেসব দল [MASK] রয়েছে , বাঝি",বাঝি,বাকি
1631207,পরে চিফ জুডিশিয়াল ম্যাজিস্ট্রেট [MASK] কবির সর...,হুমাধুন,হুমায়ুন
1655392,আর সে [MASK] জাভির সবচেয়ে বড় ভরসা হতে পারতেন ফ...,ন্য,জন্য
740022,এখন [MASK] সপ্তাহে তিন মণ করে মধু উৎপাদিত হচ্ছ...,প্রতএ,প্রতি


In [19]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((1837181, 3), (204132, 3))

In [20]:
train_data = Dataset.from_pandas(train_df.sample(1837181))
test_data = Dataset.from_pandas(test_df)

In [21]:
train_data.save_to_disk('train')
test_data.save_to_disk('test')

In [22]:
max_len = 128

In [23]:
model_name = 'flax-community/bengali-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
print(tokenizer.special_tokens_map.keys())

dict_keys(['eos_token', 'unk_token', 'pad_token', 'additional_special_tokens'])


In [25]:
tokenizer.decode(tokenizer(train_data[0]['input'],padding="max_length", max_length=64,return_attention_mask = True).input_ids)

'কিন্তু খামারির কাছ থেকে [mask] ও ব্যবসায়ীরা ব্রয়লার মুরগি কিনছেন টাকা ও সোনালি মুরগি টাকায়, পাইওঁার</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [26]:
def prepare_data(example):
    example["input_ids"] = tokenizer(example['input'], padding="max_length", max_length=64,return_attention_mask = True).input_ids
    example["attention_mask"] = tokenizer(example['input'], padding="max_length", max_length=64,return_attention_mask = True).attention_mask
    example["labels"] = tokenizer(example["output"], padding="max_length", max_length=64).input_ids
    return example

In [27]:
train_data = train_data.map(prepare_data,remove_columns=train_data.column_names)

  0%|          | 0/1837181 [00:00<?, ?ex/s]

In [28]:
test_data = test_data.map(prepare_data)

  0%|          | 0/204132 [00:00<?, ?ex/s]

In [29]:
train_data.cleanup_cache_files()
test_data.cleanup_cache_files()

0

In [30]:
train_data.save_to_disk('train')
test_data.save_to_disk('test')

In [31]:
!rm -rf /kaggle/working/data_wav2vec2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
